<a href="https://colab.research.google.com/github/akshitt/AL-OOD/blob/main/al_for_ood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Deleting files in the Trash that is occupying space

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
my_drive = GoogleDrive(gauth)

for a_file in my_drive.ListFile({'q': "trashed = true"}).GetList():
    # print the name of the file being deleted.
    print({a_file['title']},"is about to get deleted permanently.")
    # delete the file permanently.
    a_file.Delete()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!du -h --max-depth 3 /content/drive/Shareddrives
!du -hs /content/drive/Shareddrives


du: cannot read directory '/content/drive/Shareddrives/Deep_Active_Learning/camelyon17_v1.0/patches/patient_096_node_0': Input/output error
du: cannot read directory '/content/drive/Shareddrives/Deep_Active_Learning/nihdataset1': Input/output error
53G	/content/drive/Shareddrives


In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/al4ood/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!ls /content/drive/Shareddrives/Deep_Active_Learning/nihdataset1 | wc -l

88272


In [ ]:
#!cd /content/drive/Shareddrives/Deep_Active_Learning/nihfromkaggle
#!unzip "/content/drive/Shareddrives/Deep_Active_Learning/*.zip"
#from zipfile import ZipFile
#!rm -r /content/drive/Shareddrives/Deep_Active_Learning/nihfromkaggle

import os
import shutil
import zipfile
from pathlib import Path


my_dir = r"/content/drive/Shareddrives/Deep_Active_Learning/nihdataset1"
my_zip = r"/content/drive/Shareddrives/Deep_Active_Learning/data.zip"

with zipfile.ZipFile(my_zip) as zip_file:
    for member in zip_file.namelist():
        filename = os.path.basename(member)
        # skip directories
        if not filename:
            continue

        # copy file (taken from zipfile's extract)
        source = zip_file.open(member)

        if not Path(os.path.join(my_dir, filename)).is_file():
          target = open(os.path.join(my_dir, filename), "wb")
          with source, target:
            shutil.copyfileobj(source, target)
          target.close()

        source.close()

#!du -h --max-depth=2 /content/drive/Shareddrives/


BadZipFile: ignored

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import torchvision.models as models
import torchvision.transforms as trns
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import random_split, DataLoader, Dataset
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from sklearn.utils import shuffle
import h5py
import time
import random
import datetime
import copy
import numpy as np
import csv
import json
import subprocess
import sys
import PIL.Image as Image
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.models as models
from matplotlib import pyplot as plt
from distil.distil.utils.models.resnet import ResNet18
from trust.trust.utils.custom_dataset import load_dataset_custom
from torch.utils.data import Subset
from torch.autograd import Variable
import tqdm
from math import floor
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from distil.distil.active_learning_strategies.scmi import SCMI
from distil.distil.active_learning_strategies.smi import SMI
from distil.distil.active_learning_strategies.badge import BADGE
from distil.distil.active_learning_strategies.entropy_sampling import EntropySampling
from distil.distil.active_learning_strategies.gradmatch_active import GradMatchActive
seed=42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
from distil.distil.utils.utils import *


from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Distil related libraries

In [ ]:
!pip3 install sphinxcontrib-napoleon
!pip3 install sphinxcontrib-bibtex
!pip3 install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ submodlib
%cd /content/drive/Shareddrives/Deep_Active_Learning/
!git clone https://github.com/decile-team/distil.git



ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/comm

In [ ]:
import sys
sys.path.append('/content/drive/Shareddrives/Deep_Active_Learning/distil')
    from distil.utils.models.resnet import ResNet18

ModuleNotFoundError: ignored

# Map images to their location

# Load Dataset

In [ ]:
AllClasses = {
    'Atelectasis':0,
    'Consolidation':1,
    'Infiltration':2,
    'Edema':3,
    'Emphysema':4,
    'Fibrosis':5,
    'Effusion':6,
    'Pneumonia':7,
    'Pleural_Thickening':8,
    'Hernia':9,
    'Cardiomegaly':10,
    'Nodule':11,
    'Mass':12,
    'Pneumothorax':13,
    'No Finding':14
}

InClasses = {
    'Atelectasis':0,
    'Consolidation':1,
    'Infiltration':2,
    'Edema':3,
    'Emphysema':4,
    'Fibrosis':5,
    'Effusion':6,
    'Pneumonia':7,
    'Pleural_Thickening':8,
    'Hernia':9
}

OutClasses={
    'Cardiomegaly':10,
    'Nodule':11,
    'Mass':12,
    'Pneumothorax':13
}

path = '/content/drive/Shareddrives/Deep_Active_Learning/chestxray/'

train = open(f'{path}train_val_list.txt')
test = open(f'{path}test_list.txt')
train_img = train.read().split()
test_img = test.read().split()
train.close()
test.close()

# for i in range(12):
#     path = f'{path}images_' + ('00' if i < 9 else '0') + str(i + 1) + '/images/'
#     print(path)
#     for f in os.listdir(path):
#         if os.path.isfile(os.path.join(path, f)) and f in train_img:
#             os.rename(os.path.join(path, f), os.path.join("/content/drive/Shareddrives/Deep_Active_Learning/chestxray/all_images/train",f))
#         if os.path.isfile(os.path.join(path, f)) and f in test_img:
#             os.rename(os.path.join(path, f), os.path.join("/content/drive/Shareddrives/Deep_Active_Learning/chestxray/all_images/test",f))

df = pd.read_csv(f'{path}Data_Entry_2017.csv')
df.head()

num_train = 100
num_test = 1000

train = []
for i in train_img[-1*num_train:]:
  #print(i)
  label = df.loc[df['Image Index'] == i,'Finding Labels'].values[0]
  if len(label.split('|'))==1:
      train.append([i,AllClasses[label]])
  else:
      continue
test = []
for i in test_img[-1*num_test:]:
  label = df.loc[df['Image Index'] == i,'Finding Labels'].values[0]
  if len(label.split('|'))==1:
      test.append([i,AllClasses[label]])
  else:
      continue

fullset=pd.DataFrame(train, columns = ['Image', 'Label'])
df_train = pd.DataFrame(train, columns = ['Image', 'Label'])
df_test = pd.DataFrame(test, columns = ['Image', 'Label'])



In [ ]:
class ChestXray(Dataset):
    def __init__(self, data, root='/content/drive/Shareddrives/Deep_Active_Learning/chestxray/all_images', transform=None):
        self.root = root
        self.image_paths = [os.path.join(root,s) for s in data['Image']]

        self.targets = data['Label'].tolist()
        self.transform = transform


    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = np.array(Image.open(self.image_paths[idx]).convert('RGB'))
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label

In [ ]:
xray_dataset = ChestXray(data=df_test)
print(xray_dataset[1][0].shape)

## Helper Functions

In [ ]:
def create_ood_data(fullset, testset, split_cfg, num_cls, augVal):

    np.random.seed(42)
    train_idx = []
    val_idx = []
    lake_idx = []
    test_idx = []
    selected_classes = np.array(list(range(split_cfg['num_cls_idc'])))
    for i in range(num_cls): #all_classes
        full_idx_class = list(torch.where(torch.Tensor(fullset.targets) == i)[0].cpu().numpy())
        if(i in selected_classes):
            test_idx_class = list(torch.where(torch.Tensor(testset.targets) == i)[0].cpu().numpy())
            test_idx += test_idx_class
            class_train_idx = list(np.random.choice(np.array(full_idx_class), size=split_cfg['per_idc_train'], replace=False))
            train_idx += class_train_idx
            remain_idx = list(set(full_idx_class) - set(class_train_idx))
            class_val_idx = list(np.random.choice(np.array(remain_idx), size=split_cfg['per_idc_val'], replace=False))
            remain_idx = list(set(remain_idx) - set(class_val_idx))
            class_lake_idx = list(np.random.choice(np.array(remain_idx), size=split_cfg['per_idc_lake'], replace=False))
        else:
            class_train_idx = list(np.random.choice(np.array(full_idx_class), size=split_cfg['per_ood_train'], replace=False)) #always 0
            remain_idx = list(set(full_idx_class) - set(class_train_idx))
            class_val_idx = list(np.random.choice(np.array(remain_idx), size=split_cfg['per_ood_val'], replace=False)) #Only for CG ood val has samples
            remain_idx = list(set(remain_idx) - set(class_val_idx))
            class_lake_idx = list(np.random.choice(np.array(remain_idx), size=split_cfg['per_ood_lake'], replace=False)) #many ood samples in lake

        if(augVal and (i in selected_classes)): #augment with samples only from the imbalanced classes
            train_idx += class_val_idx
        val_idx += class_val_idx
        lake_idx += class_lake_idx

    train_set = SubsetWithTargets(fullset, train_idx, torch.Tensor(fullset.targets)[train_idx])
    val_set = SubsetWithTargets(fullset, val_idx, torch.Tensor(fullset.targets)[val_idx])
    lake_set = SubsetWithTargets(fullset, lake_idx, getOODtargets(torch.Tensor(fullset.targets)[lake_idx], selected_classes, split_cfg['num_cls_idc']))
    test_set = SubsetWithTargets(testset, test_idx, torch.Tensor(testset.targets)[test_idx])

    return train_set, val_set, test_set, lake_set, selected_classes

In [ ]:
split_cfg = {
    'per_imbclass_train': int(),
    'per_imbclass_val': int(),
    'per_imbclass_lake': int(),
    'per_class_train': int(1),
    'per_class_val': int(1),
    'per_class_lake': int(5),
    'sel_cls_idx': list(),
    'train_size': int(),
    'val_size': int(),
    'lake_size': int(),
    'num_rep': int(),
    'lake_subset_repeat_size': int(),
    'num_cls_imbalance': int(),
    'num_cls_idc': int(10),
    'per_idc_train': int(1),
    'per_idc_val': int(1),
    'per_idc_lake': int(5),
    'per_ood_train': int(1),
    'per_ood_val': int(1),
    'per_ood_lake': int(5)
}


In [ ]:
create_ood_data(df_train, df_test, split_cfg, num_cls=10, augVal=False)

## SMI AL Loop

In [ ]:
def model_eval_loss(data_loader, model, criterion):
    total_loss = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(data_loader):
            inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss

def init_weights(m):
#     torch.manual_seed(35)
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform_(m.weight)
    elif isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

def weight_reset(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()

def create_model(name, num_cls, device, embedding_type):
    if name == 'ResNet18':
        if embedding_type == "gradients":
            model = ResNet18(num_cls)
        else:
            model = models.resnet18()
    elif name == 'MnistNet':
        model = MnistNet()
    elif name == 'ResNet164':
        model = ResNet164(num_cls)
    model.apply(init_weights)
    model = model.to(device)
    return model

def loss_function():
    criterion = nn.CrossEntropyLoss()
    criterion_nored = nn.CrossEntropyLoss(reduction='none')
    return criterion, criterion_nored

def optimizer_with_scheduler(model, num_epochs, learning_rate, m=0.9, wd=5e-4):
    optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                          momentum=m, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    return optimizer, scheduler

def optimizer_without_scheduler(model, learning_rate, m=0.9, wd=5e-4):
#     optimizer = optim.Adam(model.parameters(),weight_decay=wd)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                          momentum=m, weight_decay=wd)
    return optimizer

def generate_cumulative_timing(mod_timing):
    tmp = 0
    mod_cum_timing = np.zeros(len(mod_timing))
    for i in range(len(mod_timing)):
        tmp += mod_timing[i]
        mod_cum_timing[i] = tmp
    return mod_cum_timing/3600

def find_err_per_class(test_set, val_set, final_val_classifications, final_val_predictions, final_tst_classifications,
                       final_tst_predictions, saveDir, prefix):
    #find queries from the validation set that are erroneous
#     saveDir = os.path.join(saveDir, prefix)
#     if(not(os.path.exists(saveDir))):
#         os.mkdir(saveDir)
    val_err_idx = list(np.where(np.array(final_val_classifications) == False)[0])
    tst_err_idx = list(np.where(np.array(final_tst_classifications) == False)[0])
    val_class_err_idxs = []
    tst_err_log = []
    val_err_log = []
    for i in range(num_cls):
        if(feature=="ood"): tst_class_idxs = list(torch.where(torch.Tensor(test_set.targets.float()) == i)[0].cpu().numpy())
        if(feature=="classimb"): tst_class_idxs = list(torch.where(torch.Tensor(test_set.targets) == i)[0].cpu().numpy())
        val_class_idxs = list(torch.where(torch.Tensor(val_set.targets.float()) == i)[0].cpu().numpy())
        #err classifications per class
        val_err_class_idx = set(val_err_idx).intersection(set(val_class_idxs))
        tst_err_class_idx = set(tst_err_idx).intersection(set(tst_class_idxs))
        if(len(val_class_idxs)>0):
            val_error_perc = round((len(val_err_class_idx)/len(val_class_idxs))*100,2)
        else:
            val_error_perc = 0
        tst_error_perc = round((len(tst_err_class_idx)/len(tst_class_idxs))*100,2)
        print("val, test error% for class ", i, " : ", val_error_perc, tst_error_perc)
        val_class_err_idxs.append(val_err_class_idx)
        tst_err_log.append(tst_error_perc)
        val_err_log.append(val_error_perc)
    tst_err_log.append(sum(tst_err_log)/len(tst_err_log))
    val_err_log.append(sum(val_err_log)/len(val_err_log))
    return tst_err_log, val_err_log, val_class_err_idxs


def aug_train_subset(train_set, lake_set, true_lake_set, subset, lake_subset_idxs, budget, augrandom=False):
    all_lake_idx = list(range(len(lake_set)))
    if(not(len(subset)==budget) and augrandom):
        print("Budget not filled, adding ", str(int(budget) - len(subset)), " randomly.")
        remain_budget = int(budget) - len(subset)
        remain_lake_idx = list(set(all_lake_idx) - set(subset))
        random_subset_idx = list(np.random.choice(np.array(remain_lake_idx), size=int(remain_budget), replace=False))
        subset += random_subset_idx
    lake_ss = SubsetWithTargets(true_lake_set, subset, torch.Tensor(true_lake_set.targets.float())[subset])
    if(feature=="ood"):
        ood_lake_idx = list(set(lake_subset_idxs)-set(subset))
        private_set =  SubsetWithTargets(true_lake_set, ood_lake_idx, torch.Tensor(np.array([split_cfg['num_cls_idc']]*len(ood_lake_idx))).float())
    remain_lake_idx = list(set(all_lake_idx) - set(lake_subset_idxs))
    remain_lake_set = SubsetWithTargets(lake_set, remain_lake_idx, torch.Tensor(lake_set.targets.float())[remain_lake_idx])
    remain_true_lake_set = SubsetWithTargets(true_lake_set, remain_lake_idx, torch.Tensor(true_lake_set.targets.float())[remain_lake_idx])
    print(len(lake_ss),len(remain_lake_set),len(lake_set))
    if(feature!="ood"): assert((len(lake_ss)+len(remain_lake_set))==len(lake_set))
    aug_train_set = torch.utils.data.ConcatDataset([train_set, lake_ss])
    if(feature=="ood"):
        return aug_train_set, remain_lake_set, remain_true_lake_set, private_set, lake_ss
    else:
        return aug_train_set, remain_lake_set, remain_true_lake_set, lake_ss

def getQuerySet(val_set, val_class_err_idxs, imb_cls_idx, miscls):
    miscls_idx = []
    if(miscls):
        for i in range(len(val_class_err_idxs)):
            if i in imb_cls_idx:
                miscls_idx += val_class_err_idxs[i]
        print("total misclassified ex from imb classes: ", len(miscls_idx))
    else:
        for i in imb_cls_idx:
            imb_cls_samples = list(torch.where(torch.Tensor(val_set.targets.float()) == i)[0].cpu().numpy())
            miscls_idx += imb_cls_samples
        print("total samples from imb classes as targets: ", len(miscls_idx))
    return Subset(val_set, miscls_idx)


def getPrivateSet(lake_set, subset, private_set):
    #augment prev private set and current subset
    new_private_set = SubsetWithTargets(lake_set, subset, torch.Tensor(lake_set.targets.float())[subset])
#     new_private_set =  Subset(lake_set, subset)
    total_private_set = torch.utils.data.ConcatDataset([private_set, new_private_set])
    return total_private_set

def remove_ood_points(lake_set, subset, idc_idx):
    idx_subset = []
    subset_cls = torch.Tensor(lake_set.targets.float())[subset]
    for i in idc_idx:
        idc_subset_idx = list(torch.where(subset_cls == i)[0].cpu().numpy())
        idx_subset += list(np.array(subset)[idc_subset_idx])
    print(len(idx_subset),"/",len(subset), " idc points.")
    return idx_subset

def getPerClassSel(lake_set, subset, num_cls):
    perClsSel = []
    subset_cls = torch.Tensor(lake_set.targets.float())[subset]
    for i in range(num_cls):
        cls_subset_idx = list(torch.where(subset_cls == i)[0].cpu().numpy())
        perClsSel.append(len(cls_subset_idx))
    return perClsSel

In [ ]:
feature = "ood"
device_id = 0
run="fkna_3"
datadir = 'data/'
data_name = 'ChestXray'
model_name = 'ResNet18'
num_rep = 10
learning_rate = 0.01
num_runs = 1  # number of random runs
computeClassErrorLog = True

magnification = 1
device = "cuda:"+str(device_id) if torch.cuda.is_available() else "cpu"
datkbuildPath = "./datk/build"
exePath = "cifarSubsetSelector"
print("Using Device:", device)
doublePrecision = True
linearLayer = True
miscls = False
# handler = DataHandler_CIFAR10
augTarget = True
embedding_type = "gradients"

if(feature=="ood"):
    num_cls=8
    budget=250
    num_epochs = int(10)
    split_cfg = {'num_cls_idc':8, 'per_idc_train':200, 'per_idc_val':10, 'per_idc_lake':500, 'per_ood_train':0, 'per_ood_val':0, 'per_ood_lake':5000}#cifar10
    # split_cfg = {'num_cls_idc':50, 'per_idc_train':100, 'per_idc_val':2, 'per_idc_lake':100, 'per_ood_train':0, 'per_ood_val':0, 'per_ood_lake':500}#cifar100
    initModelPath = "weights/" + data_name + "_" + feature + "_" + model_name + "_" + str(learning_rate) + "_" + str(split_cfg["per_idc_train"]) + "_" + str(split_cfg["per_idc_val"]) + "_" + str(split_cfg["num_cls_idc"])